In [1]:
!pip install cmake dlib face_recognition numpy

## Importing Libraries

In [1]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime 

In [2]:
os.getcwd()

'c:\\Users\\dell\\Downloads'

 ## Path For Test Images

In [7]:

# Define the main directory containing the training images
base_dir = r"C:\Users\dell\Downloads\Training images"
images = []
classNames = []

# Loop through each folder (each person's name)
for person_folder in os.listdir(base_dir):
    person_path = os.path.join(base_dir, person_folder)
    
    # Check if it's a directory (i.e., a folder containing images of one person)
    if os.path.isdir(person_path):
        print(f"Processing images for: {person_folder}")
        
        # Loop through images in the person's folder
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            
            # Read the image and add it to the list
            curImg = cv2.imread(img_path)
            if curImg is not None:  # Make sure the image is read correctly
                images.append(curImg)
                classNames.append(person_folder)  # Use folder name as the class label

print(classNames)

Processing images for: Abdlrahman Elshaer
Processing images for: Moaz Tawfik
Processing images for: Nada Medhat
Processing images for: Nedaa Elsherbini
Processing images for: Noha Elbilkasy
['Abdlrahman Elshaer', 'Abdlrahman Elshaer', 'Abdlrahman Elshaer', 'Abdlrahman Elshaer', 'Abdlrahman Elshaer', 'Moaz Tawfik', 'Moaz Tawfik', 'Moaz Tawfik', 'Moaz Tawfik', 'Moaz Tawfik', 'Nada Medhat', 'Nada Medhat', 'Nada Medhat', 'Nada Medhat', 'Nada Medhat', 'Nada Medhat', 'Nedaa Elsherbini', 'Nedaa Elsherbini', 'Nedaa Elsherbini', 'Nedaa Elsherbini', 'Nedaa Elsherbini', 'Noha Elbilkasy', 'Noha Elbilkasy', 'Noha Elbilkasy', 'Noha Elbilkasy']


## Extracting names from the List


 ## Converting the Images into Encodings


In [8]:
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList


## Accessing Camera

In [ ]:
# Capture video from webcam
cap = cv2.VideoCapture(0)
frame_resizing = 0.25

# Get the known encodings
encodeListKnown = findEncodings(images)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break

    imgS = cv2.resize(img, (0, 0), fx=frame_resizing, fy=frame_resizing)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    # Find faces and encodings in the current frame
    facesCurFrame = face_recognition.face_locations(imgS)
    encodesCurFrame = face_recognition.face_encodings(imgS, facesCurFrame)

    for encodeFace, faceLoc in zip(encodesCurFrame, facesCurFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        print('Matches:', matches)
        
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        print('Face distances:', faceDis)
        
        matchIndex = np.argmin(faceDis)
        
        if matches[matchIndex]:
            name = classNames[matchIndex].upper()
            print(name)
            
            # Scale face location back to original image size
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = int(y1 / frame_resizing), int(x2 / frame_resizing), int(y2 / frame_resizing), int(x1 / frame_resizing)

            # Draw rectangle around the face and display the name
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            cv2.putText(img, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            # markAttendance(name)  # Uncomment this line if you have the function defined

    cv2.imshow('Webcam', img)
    key = cv2.waitKey(1)
    if key == 27:  # Press 'ESC' to exit
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
#Let's test the code

test_images_dir = "test_images/"
correct_predictions = 0
total_predictions = 0

# Loop through each person's folder in the test set
for person_folder in os.listdir(test_images_dir):
    person_path = os.path.join(test_images_dir, person_folder)

    # Check if it's a directory (i.e., a folder containing images of one person)
    if os.path.isdir(person_path):
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            
            # Load and process the test image
            img = cv2.imread(img_path)
            imgS = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
            imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
            
            # Detect faces and encode them
            face_locations = face_recognition.face_locations(imgS)
            encodesCurFrame = face_recognition.face_encodings(imgS, face_locations)

            for encodeFace in encodesCurFrame:
                matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
                faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
                matchIndex = np.argmin(faceDis)
                
                # Predicted name
                if matches[matchIndex]:
                    predicted_name = classNames[matchIndex].upper()
                else:
                    predicted_name = "Unknown"

                # Actual name
                actual_name = person_folder.upper()

                # Compare actual name with predicted name
                if predicted_name == actual_name:
                    correct_predictions += 1

                total_predictions += 1

# Calculate accuracy
if total_predictions > 0:
    accuracy = (correct_predictions / total_predictions) * 100
    print(f"Model Accuracy: {accuracy:.2f}%")
else:
    print("No test images found.")